In [1]:
import pandas as pd
df = pd.read_csv('input/example.csv')
df = df[df['Weather'].notna()] # remove any NaN values as it blows up serialization
data = df.to_dict('records') # Get only 700 records. More records will make it slower to index
len(data)
data

[{'Weather': 'mild no clouds'},
 {'Weather': 'cloudy windy'},
 {'Weather': 'sunny hot'},
 {'Weather': 'storm'}]

In [2]:
from qdrant_client import models, QdrantClient
from sentence_transformers import SentenceTransformer

c:\Users\evgen\miniconda3\envs\genai\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
encoder = SentenceTransformer('all-MiniLM-L6-v2') # Model to create embeddings

In [4]:
# create the vector database client
qdrant = QdrantClient(":memory:") # Create in-memory Qdrant instance

In [5]:
# Create collection to store wines
qdrant.recreate_collection(
    collection_name="weather",
    vectors_config=models.VectorParams(
        size=encoder.get_sentence_embedding_dimension(), # Vector size is defined by used model
        distance=models.Distance.COSINE
    )
)

C:\Users\evgen\AppData\Local\Temp\ipykernel_15488\1843291466.py:2: DeprecationWarning: `recreate_collection` method is deprecated and will be removed in the future. Use `collection_exists` to check collection existence and `create_collection` instead.
  qdrant.recreate_collection(


True

In [6]:
# vectorize!
qdrant.upload_points(
    collection_name="weather",
    points=[
        models.PointStruct(
            id=idx,
            vector=encoder.encode(doc["Weather"]).tolist(),
            payload=doc,
        ) for idx, doc in enumerate(data) # data is the variable holding all the wines
    ]
)

In [7]:
user_prompt = "Suggest me a good weather for walk outside"

In [8]:
# Search time for awesome wines!

hits = qdrant.search(
    collection_name="weather",
    query_vector=encoder.encode(user_prompt).tolist(),
    limit=3
)
for hit in hits:
  print(hit.payload, "score:", hit.score)

{'Weather': 'cloudy windy'} score: 0.4282032121559556
{'Weather': 'mild no clouds'} score: 0.38260891911840944
{'Weather': 'sunny hot'} score: 0.3408248706452164


In [9]:
# define a variable to hold the search results
search_results = [hit.payload for hit in hits]

In [11]:
# Create a server using llamafile

# cd models
# phi-2.llamafile -ngl 9999

'phi-2.llamafile.exe' is not recognized as an internal or external command,
operable program or batch file.


In [12]:
from openai import OpenAI
client = OpenAI(
    base_url="http://localhost:8080/v1", #make sure that Llamafile server is running on 8080
    api_key = "no-key-required"
)

In [ ]:
# send request to the server, save response
completion = client.chat.completions.create(
    model="LLaMA_CPP",
    messages=[
        {"role": "system", "content": "You AI assistant, use only words from content to answer the user's question."},
        {"role": "user", "content": user_prompt},
        {"role": "assistant", "content": str(search_results)}
    ]
)

In [17]:
print(completion.choices[0].message.content)

You can go for a walk outside if the weather is mild with no clouds.

